In [1]:
import os, sys, ujson, json, pandas as pd, numpy as np
from sklearn.feature_selection import VarianceThreshold
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from Feature.common import *

# 筛选 feature

## 检查缺失值情况

In [17]:
raw_df = cPickle.load(open('raw_feature.dat', 'rb'))

In [22]:
# 用 20181016 的数据(有问题的数据)
保留输出不运行！！
for col in raw_df.columns:
    missing = len(raw_df) - np.count_nonzero( raw_df[col].isnull().values)
    mis_perc = 100 - float(missing) / len(raw_df) * 100
    
    if mis_perc > 50:
        print ("{miss} %\t {col}".format( col=col,miss=round(mis_perc,2)))

99.65 %	 push_category
99.65 %	 push_domain
99.65 %	 push_keywords
99.85 %	 push_subcategory
99.77 %	 push_supervised_keywords
99.65 %	 push_title_keywords
99.77 %	 push_topic
99.77 %	 push_topic2048
99.67 %	 push_topic256
99.65 %	 push_topic64


In [53]:
# 用20180901的数据
print('missing ratio:\n')
for col in raw_df.columns:
    missing = np.count_nonzero( raw_df[col].isnull().values)
    mis_perc = float(missing) / len(raw_df) * 100
    
    if mis_perc :
        print ("{miss} %\t {col}".format(col=col,miss=round(mis_perc,2)))

missing ratio:

32.04 %	 category_v2_cross
32.04 %	 category_v2_score
6.36 %	 disgusting_scores
6.36 %	 image_nsfw_scores
32.62 %	 key_entities
5.12 %	 key_entities_v2
8.23 %	 key_entities_v2_hash
42.2 %	 keywords_tag
32.04 %	 keywords_v2
6.36 %	 max_disgusting_scores
1.09 %	 nl_category
0.37 %	 nl_domain
8.29 %	 nl_subcategory
29.42 %	 nl_supervised_keywords
0.33 %	 nl_title_keywords
29.61 %	 nl_topic
29.45 %	 nl_topic2048
0.1 %	 opera_id
6.15 %	 pictures
4.1 %	 publication_time
54.64 %	 push_category
54.94 %	 push_domain
54.98 %	 push_keywords
63.53 %	 push_subcategory
68.33 %	 push_supervised_keywords
54.69 %	 push_title_keywords
68.79 %	 push_topic
68.74 %	 push_topic2048
54.93 %	 push_topic256
54.97 %	 push_topic64
32.04 %	 supervised_keywords
32.96 %	 supervised_keywords_v2
32.96 %	 supervised_keywords_v2_origin
0.08 %	 title_keywords
32.04 %	 topic
32.04 %	 topic2048
32.04 %	 topic_v2


>后一个cell 计算出的missing ratio 是基于 20181016日算的，前一个cell算的是20180901. 能明显看出，后者 push_ 相关feature缺省更为严重，这是因为系统bug

>剔除除了push以外的相关缺省严重的 feature。设 threshold = 50%。缺省一半以上的feature, 除了 push_相关 feature 都扔。

## discard unqiue columns

> 筛选 unqiue value 太多的，直接扔。筛选方差不大的，直接扔。最后 impute

In [2]:
raw_df = cPickle.load(open('raw_feature.dat', 'rb'))

In [3]:
# drop missing ratio 大的feature以后的数据：
unique_cols = []

for col in raw_df.columns:
    if isinstance( raw_df[col][0], dict) or isinstance(raw_df[col][0], list):
        continue
    
    values = raw_df[col]
    unique_values = [i for i in values if i != None and i != '']
    
    if unique_values:
        unique_perct = float(len(set(unique_values)))/len(unique_values)*100
    
        if unique_perct > 20:
            print ("{col}: \t\t{miss}%".format(col=col,miss=round(unique_perct,2)))

news_device_id: 		30.74%
opera_id: 		30.68%
response_timestamp: 		71.23%
summary: 		66.67%


暂且保留：response_timestamp，想做 normalizing, scaling。 其他三个feature，丢～

## discard var ->0
> 筛选方差， 方差趋近于零的，丢～

In [2]:
raw_df = cPickle.load(open('raw_feature.dat', 'rb'))

In [12]:
# 找到所有 numerical cols
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

newdf = raw_df.select_dtypes(include=numerics)

for c in newdf.columns:
    var = np.var( newdf[c].values )
    
    print ("{var}\t, {col}".format(var = round(var, 2), col = c))

18757774.08	, content_length
0.02	, evergreen_confidence
0.24	, label
35129605.92	, news_publish_time_diff_hour_desc
1.94	, no_of_pictures
0.08	, no_of_videos
4.55262942139e+14	, response_timestamp
199151.2	, screen_height
65920.91	, screen_width
2.17	, soure_num
0.91	, spam_word_count
2018903791.94	, timestamp
1186492338.1	, ttl
471739.5	, word_count


丢方差太小的： evergreen_confidence, no_of_videos

## 

# 研究每个feature的分布
>筛除 std = 0 的feature 以后，还剩 129个 feature

In [15]:
raw_feature = cPickle.load(open('raw_feature.dat', 'rb'))
raw_feature_df = pd.DataFrame(raw_feature)

In [19]:
raw_feature_df = raw_feature_df.drop(['low_taste_keywords', 'quality', 'sanitized_html_length', 'score'], axis = 1)

In [20]:
raw_feature_df.describe()

,content_length,evergreen_confidence,label,last_timestamp,new_type,news_publish_time_diff_hour_desc,no_of_pictures,no_of_videos,response_timestamp,screen_height,screen_width,soure_num,spam_word_count,timestamp,ttl,word_count
count,32772.000000,32772.000000,32772.000000,3.277200e+04,15485.000000,32772.000000,32772.000000,32772.000000,3.277200e+04,32772.000000,32772.000000,32772.000000,32772.000000,3.277200e+04,3.277200e+04,32772.000000
mean,958.282284,0.071739,0.379470,1.539656e+09,1.062254,-301.250946,1.227908,0.083608,1.538593e+09,1211.517545,706.816673,1.467808,0.225803,1.539676e+09,1.555234e+09,327.127426
std,822.979503,0.132962,0.485263,2.048305e+05,0.241624,11334.065384,1.091308,0.303113,4.080254e+07,430.499270,250.331768,1.668263,0.669047,2.252041e+04,1.777342e+04,284.594212
min,103.000000,0.000000,0.000000,1.532504e+09,1.000000,-427101.000000,0.000000,0.000000,2.147483e+06,320.000000,240.000000,1.000000,0.000000,1.539518e+09,1.555152e+09,45.000000
25%,441.000000,0.000000,0.000000,1.539667e+09,1.000000,5.000000,1.000000,0.000000,1.539699e+09,854.000000,480.000000,1.000000,0.000000,1.539670e+09,1.555226e+09,140.000000
50%,698.000000,0.000000,0.000000,1.539677e+09,1.000000,8.000000,1.000000,0.000000,1.539709e+09,1204.000000,720.000000,1.000000,0.000000,1.539678e+09,1.555232e+09,237.000000
75%,1347.000000,0.077211,1.000000,1.539688e+09,1.000000,10.000000,1.000000,0.000000,1.539718e+09,1280.000000,720.000000,1.000000,0.000000,1.539689e+09,1.555244e+09,419.000000
max,8089.000000,0.808598,1.000000,1.539722e+09,2.000000,2213.000000,125.000000,4.000000,1.547661e+09,2960.000000,2560.000000,132.000000,14.000000,1.539722e+09,1.555284e+09,2951.000000


In [23]:
len(raw_feature_df.keys())

129